# Tools and Routing

In [1]:
from openai import OpenAI
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

client = OpenAI()

In [2]:
from langchain.agents import tool

In [3]:
@tool
def search(query: str) -> str:
    """Search for weather online"""
    return "42f"

In [4]:
search.name

'search'

In [5]:
search.description

'Search for weather online'

In [6]:
search.args

{'query': {'title': 'Query', 'type': 'string'}}

In [11]:
from langchain_core.pydantic_v1 import BaseModel, Field
class SearchInput(BaseModel):
    query: str = Field(description="Thing to search for")


In [12]:
@tool(args_schema=SearchInput)
def search(query: str) -> str:
    """Search for the weather online."""
    return "42f"

In [13]:
search.args

{'query': {'title': 'Query',
  'description': 'Thing to search for',
  'type': 'string'}}

In [14]:
search.run("sf")

'42f'

In [15]:
import requests
from langchain_core.pydantic_v1 import BaseModel, Field
from datetime import datetime, timezone

# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
        
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    # Get the current UTC time and convert it to the local timezone
    current_utc_time = datetime.now(timezone.utc).astimezone()

    # Parse the times in results['hourly']['time'] assuming they are in ISO 8601 format with 'Z' for UTC
    time_list = [datetime.fromisoformat(time_str.replace('Z', '+00:00')).astimezone() for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°C'

In [16]:
get_current_temperature.name

'get_current_temperature'

In [17]:
get_current_temperature.description

'Fetch current temperature for given coordinates.'

In [18]:
get_current_temperature.args

{'latitude': {'title': 'Latitude',
  'description': 'Latitude of the location to fetch weather data for',
  'type': 'number'},
 'longitude': {'title': 'Longitude',
  'description': 'Longitude of the location to fetch weather data for',
  'type': 'number'}}

In [19]:
from langchain_core.utils.function_calling import convert_to_openai_function

In [20]:
convert_to_openai_function(get_current_temperature)

{'name': 'get_current_temperature',
 'description': 'Fetch current temperature for given coordinates.',
 'parameters': {'type': 'object',
  'properties': {'latitude': {'description': 'Latitude of the location to fetch weather data for',
    'type': 'number'},
   'longitude': {'description': 'Longitude of the location to fetch weather data for',
    'type': 'number'}},
  'required': ['latitude', 'longitude']}}

In [21]:
get_current_temperature.invoke({"latitude": 13, "longitude": 14})

'The current temperature is 38.6°C'

In [22]:
import wikipedia
@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [23]:
search_wikipedia.name

'search_wikipedia'

In [24]:
search_wikipedia.description

'Run Wikipedia search and get page summaries.'

In [25]:
convert_to_openai_function(search_wikipedia)

{'name': 'search_wikipedia',
 'description': 'Run Wikipedia search and get page summaries.',
 'parameters': {'type': 'object',
  'properties': {'query': {'type': 'string'}},
  'required': ['query']}}

In [26]:
search_wikipedia.invoke({"query": "langchain"})

"Page: LangChain\nSummary: LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\n\n\nPage: DataStax\nSummary: DataStax, Inc. is a real-time data for AI company based in Santa Clara, California. Its product Astra DB is a cloud database-as-a-service based on Apache Cassandra. DataStax also offers DataStax Enterprise (DSE), an on-premises database built on Apache Cassandra, and Astra Streaming, a messaging and event streaming cloud service based on Apache Pulsar. As of June 2022, the company has roughly 800 customers distributed in over 50 countries.\n\n\n\nPage: Sentence embedding\nSummary: In natural language processing, a sentence embedding refers to a numeric representation of a sentence in the form of a vector of real numbers whi

### Routing

In lesson 3, we show an example of function calling deciding between two candidate functions.

Given our tools above, let's format these as OpenAI functions and show this same behavior.

In [32]:
from langchain_openai import ChatOpenAI

In [33]:
functions = [
    convert_to_openai_function(f) for f in [
        search_wikipedia, get_current_temperature
    ]
]
model = ChatOpenAI(temperature=0).bind(functions=functions)

In [34]:
model.invoke("what is the weather in sf right now")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 105, 'total_tokens': 130}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-5816e75e-9f8b-48af-b696-b8dbbf9cbac8-0', usage_metadata={'input_tokens': 105, 'output_tokens': 25, 'total_tokens': 130})

In [35]:
model.invoke("what is langchain")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"Langchain"}', 'name': 'search_wikipedia'}}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 101, 'total_tokens': 117}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-f7b5d478-9c30-4bdc-b708-c13ffa656fd1-0', usage_metadata={'input_tokens': 101, 'output_tokens': 16, 'total_tokens': 117})

In [36]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])
chain = prompt | model

In [37]:
chain.invoke({"input": "what is the weather in sf right now"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 113, 'total_tokens': 138}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-ce47fbc5-9e7b-4a3a-b1db-e940d2985814-0', usage_metadata={'input_tokens': 113, 'output_tokens': 25, 'total_tokens': 138})

In [38]:
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [39]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [40]:
result = chain.invoke({"input": "what is the weather in sf right now"})

In [41]:
type(result)

langchain_core.agents.AgentActionMessageLog

In [42]:
result.tool

'get_current_temperature'

In [43]:
result.tool_input

{'latitude': 37.7749, 'longitude': -122.4194}

In [45]:
get_current_temperature.invoke(result.tool_input)

'The current temperature is 12.2°C'

In [46]:
result = chain.invoke({"input": "hi!"})

In [47]:
type(result)

langchain_core.agents.AgentFinish

In [48]:
result.return_values

{'output': 'Well, hello there! How can I assist you today?'}

In [49]:
from langchain.schema.agent import AgentFinish
def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        tools = {
            "search_wikipedia": search_wikipedia, 
            "get_current_temperature": get_current_temperature,
        }
        return tools[result.tool].run(result.tool_input)

In [50]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser() | route

In [51]:
result = chain.invoke({"input": "What is the weather in san francisco right now?"})

In [52]:
result

'The current temperature is 12.2°C'

In [53]:
result = chain.invoke({"input": "What is langchain?"})

In [54]:
result

"Page: LangChain\nSummary: LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\n\n\nPage: DataStax\nSummary: DataStax, Inc. is a real-time data for AI company based in Santa Clara, California. Its product Astra DB is a cloud database-as-a-service based on Apache Cassandra. DataStax also offers DataStax Enterprise (DSE), an on-premises database built on Apache Cassandra, and Astra Streaming, a messaging and event streaming cloud service based on Apache Pulsar. As of June 2022, the company has roughly 800 customers distributed in over 50 countries.\n\n\n\nPage: Sentence embedding\nSummary: In natural language processing, a sentence embedding refers to a numeric representation of a sentence in the form of a vector of real numbers whi

In [55]:
chain.invoke({"input": "hi!"})

'Well, hello there! How can I assist you today?'